In [1]:
import random
random.seed(12)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch.utils.data import Dataset, ConcatDataset, SubsetRandomSampler, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import cv2

import math
import pickle

from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

## Data set

In [2]:
class FERDataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, dff, transforms):
        'Initialization'
        self.transforms = transforms
        self.dff= pd.read_csv(dff) if type(dff) is str else dff
        
        self.dff['pixels'] = [[int(y) for y in x.split()] for x in self.dff['pixels']]

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.dff)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        #ID = self.list_IDs[index]

        # Load data and get label
        X = self.dff.iloc[index]['pixels']
#         X = X.split()
        X = np.array(X, dtype=np.uint8)
        X = X.reshape(48,48)
        
        y = int(self.dff.iloc[index]['emotion'])

        if self.transforms:
            X = self.transforms(image=X)['image']

#             X = torch.cat((X,X,X),0)

        return X, y

In [3]:
batch_size= 64

df = pd.read_csv('/tf/data/Quan/fer2013/data/csv_file/fer2013.csv')

df_train = df[df['Usage'] == 'Training']
df_val = df[df['Usage'] == 'PublicTest']
df_test = df[df['Usage'] == 'PrivateTest']


train_transforms = A.Compose([
#     A.CLAHE(),
    A.Resize(48,48),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, p=0.5, border_mode=0, value=0),
#     A.RandomCrop(height=40, width=40),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2()
])


test_transforms = A.Compose([
#     A.CLAHE(),
    A.Resize(48,48),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2()
])

# train_set = FERDataset(df_train, train_transforms)
train_set = FERDataset(df_train, test_transforms) # no augmentation!
val_set = FERDataset(df_val, test_transforms)
test_set = FERDataset(df_test, test_transforms)

dataset = ConcatDataset([train_set, val_set, test_set])

# train_loader = torch.utils.data.DataLoader(train_set,
#                                              batch_size=batch_size, shuffle=True,
#                                              num_workers=8)
# val_loader = torch.utils.data.DataLoader(val_set,
#                                              batch_size=batch_size, shuffle=False,
#                                              num_workers=8)
# test_loader = torch.utils.data.DataLoader(test_set,
#                                              batch_size=batch_size, shuffle=False,
#                                              num_workers=8)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Exploss residual

In [4]:
class ExpLoss(nn.Module):
    def __init__(self, depth, residual=True):
        super(ExpLoss, self).__init__()
        
        self.depth = depth
        self.residual = residual
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        
        self.fc1 = nn.Linear(depth, depth)
        self.fc2 = nn.Linear(depth, depth)
        
#         self.main_fc = nn.Linear(depth, depth)
        
        self.sim_act = nn.Sigmoid()
        self.att_act = nn.Sigmoid()
        
        self.out_fc = nn.Sequential(nn.Linear(depth, depth),
                                    nn.BatchNorm1d(depth),
                                    nn.ReLU())
        
    def forward(self, x, labels):
        batch_size = x.size(0)
        
        x = self.avgpool(x)
        x = self.flatten(x)
        
        if self.residual:
            x_res = x
        
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        
        # cal sims (batchsize, depth)
        sims = torch.mm(x1, x2.permute(1,0))
        sims = sims / batch_size
        mask = self.filter_mask(labels)
        att = sims * (1 - mask.to(sims.device)) # only consider sample of same labels
#         att = self.att_act(sims)
#         print('att after sigmoid: ', att.min(), att.max(), att.mean(), att.var())
        
#         print('att pure: ', sims.min(), sims.max(), sims.mean(), sims.var())
#         att = torch.tanh(sims)
#         print('att after tanh: ', att.min(), att.max(), att.mean(), att.var())
#         att = self.relu(att)
#         print('att after relu: ', att.min(), att.max(), att.mean(), att.var())
# #         print('min: %.4f, max: %.4f, var: %.4f' % (att.min(), att.max(), att.std()))
        
#         x = self.main_fc(x)
        x = torch.mm(att, x)
#         print('x before out_fc: ', x.min(), x.max(), x.mean(), x.var())
        x = self.out_fc(x)
    
        if self.residual:
            x = x + x_res
    
#         print('x after out_fc: ', x.min(), x.max(), x.mean(), x.var())
        
#         print('att: ', att.min(), att.max(), att.mean(), att.var())
        
        
#         # cal sims loss
#         mask = self.filter_mask(labels)
#         sims = sims * mask.to(sims.device)


#         # prevent negative value
#         sims = self.sim_act(sims)
        
#         sims = sims.sum()
#         sims = sims / self.depth
        
        return x, 0

    def filter_mask(self, labels):
        """
        zero
        """
        classes = torch.unique(labels)
        mask = torch.ones((len(labels), len(labels)))
        
        indices = [(labels == k).nonzero().flatten() for k in classes]
        lindices = [torch.combinations(k, r=2, with_replacement=True) for k in indices]
        rindices = [torch.combinations(k.flip(0), r=2, with_replacement=True) for k in indices]
        indices = [torch.cat([lindices[i], rindices[i]]) for i in range(len(lindices))]

        for k in indices:
            mask[k[:,0], k[:,1]] = 0.
            
        return mask

In [5]:
class sVGG_exp(nn.Module):
    def __init__(self, features, in_features_classifier, n_classes):
        super(sVGG_exp, self).__init__()
        self.features = features
        self.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.exploss = ExpLoss(512)
#         self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.classifier = nn.Sequential(nn.Flatten(),
                                        nn.Linear(in_features_classifier, in_features_classifier),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(0.5, inplace=False),
                                        nn.Linear(in_features_classifier, in_features_classifier // 2),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(0.5, inplace=False),
                                        nn.Linear(in_features_classifier // 2, n_classes))
        
    def forward(self, x, labels, return_att=True):
        x = self.features(x)
        att, sim_loss = self.exploss(x, labels)

#         att = self.avgpool(x)
        x = self.classifier(att)
        
        if return_att:
            return x, sim_loss, att
        return x, sim_loss
        

## KFold

In [15]:
from sklearn.model_selection import KFold
from torchvision.models import vgg16_bn

n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True)

In [ ]:
batch_size= 64
lr = 0.001
epochs = 50
device = torch.device('cuda')
optimal_folds = [] # [test loss, test acc]

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print('Fold ', fold + 1, ' / ', n_folds, ' ********************* ')
#     print('len train_ids: ', len(train_ids))
#     print('len test_ids: ', len(test_ids))
    
    # define sub train and test
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)
    
    # data loader
    train_loader = DataLoader(dataset, batch_size, sampler=train_subsampler)
    test_loader = DataLoader(dataset, batch_size, sampler=test_subsampler)
    
#     print('len train_loader: ', len(train_loader), len(train_loader.dataset))
#     print('len test_loader: ', len(test_loader), len(test_loader.dataset))
    
    # create model
    model = sVGG_exp(features=vgg16_bn(pretrained=False).features[:34], in_features_classifier=512, n_classes=7)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    
    model_path = 'exploss_crossval/resExploss_model_fold' + str(fold+1) + '.pt'
    
    # train
    best_acc = 0.0
    curloss = -1
    hist = []
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        print('Epoch: ', epoch + 1)
        running_loss = 0.0
        running_acc = 0.0
        processed = 0.0

        # TRAIN
        model.train()
        pbar = tqdm(train_loader, position=0)
        for i, data in enumerate(pbar):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
#             print('Unique labels: ', torch.unique(labels))

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs, sim_loss, features = model(inputs, labels)
            loss = criterion(outputs, labels) + sim_loss
            acc = (torch.argmax(outputs, dim=1) == labels).float().sum()

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += (loss.item() * inputs.size(0))
            running_acc += acc.item()
            processed += len(inputs)

            pbar.set_description(desc= f'Loss={loss.item()} Batch_id={i} Accuracy={acc.item() / inputs.size(0)}')

        print('- Avg.loss: %.4f | Avg.acc: %.4f' % (running_loss / len(train_ids), running_acc / len(train_ids)))
        avgloss = running_loss / len(train_ids)
        avgacc = running_acc / len(train_ids)

        # EVALUATE
        model.eval()
        running_valloss = 0.0
        running_valacc = 0.0
        for i,data in enumerate(test_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, sim_loss, features = model(inputs, labels)

            loss = criterion(outputs, labels) + sim_loss
            acc = (torch.argmax(outputs, dim=1) == labels).float().sum()

            running_valloss += (loss.item() * inputs.size(0))
            running_valacc += acc.item()

        print('- Avg. test_loss: %.4f | Avg. test_acc: %.4f' % (running_valloss / len(test_ids), running_valacc / len(test_ids)))

        avgvalloss = running_valloss / len(test_ids)
        avgvalacc = running_valacc / len(test_ids)

        scheduler.step(avgvalacc)

        hist.append([avgloss, avgvalloss, avgacc, avgvalacc])

        if best_acc < avgvalacc:
            best_acc = avgvalacc
            curloss = avgvalloss
            torch.save(model, model_path)
            print('* Update optimal model')
    
    
    pickle.dump(hist, open('exploss_crossval/resExploss_hist_fold' + str(fold+1) + '.hdf5', 'wb'))
    optimal_folds.append([curloss, best_acc])
    print('Optimal result: test_loss: %.4f | test_acc: %.4f' % (curloss, best_acc))
    print(' - Finished Training of fold ', fold + 1, ' - ')

Fold  1  /  5  ********************* 


  0%|          | 0/449 [00:00<?, ?it/s]

Epoch:  1


Loss=0.8564144372940063 Batch_id=448 Accuracy=0.5675675675675675: 100%|██████████| 449/449 [00:39<00:00, 11.48it/s]


- Avg.loss: 1.2299 | Avg.acc: 0.4954


...

Loss=0.0610218308866024 Batch_id=448 Accuracy=1.0: 100%|██████████| 449/449 [00:38<00:00, 11.54it/s]       


- Avg.loss: 0.0415 | Avg.acc: 0.9897
- Avg. test_loss: 0.2359 | Avg. test_acc: 0.9360


Loss=0.0015432958025485277 Batch_id=0 Accuracy=1.0:   0%|          | 1/449 [00:00<01:08,  6.49it/s]

* Update optimal model
Epoch:  43


Loss=0.032886479049921036 Batch_id=376 Accuracy=1.0:  84%|████████▍ | 377/449 [00:32<00:06, 11.48it/s]     

In [ ]:
pickle.dump(optimal_folds, open('exploss_crossval/optimal_folds.hdf5', 'wb'))

In [19]:
optimal_folds

[[0.24028798388709063, 0.9281136806910003],
 [0.21653891619897944, 0.9201727500696573],
 [0.17440381800836052, 0.9423157308067438],
 [0.25318973042331433, 0.910965584506061],
 [0.2775112829517983, 0.9175142817333147]]